<a href="https://colab.research.google.com/github/tanjimmostafa/Bias-detection-bangla-word-embeddings/blob/main/Detecting_Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Bangla Word2vec Model

In [ ]:
!pip install -U bnlp_toolkit

In [ ]:
!mkdir models
%cd models

In [ ]:
!wget https://huggingface.co/sagorsarker/bangla_word2vec/resolve/main/bangla_word2vec_gen4.zip
!unzip bangla_word2vec_gen4.zip
!rm -rf bangla_word2vec_gen4.zip

In [ ]:
# prompt: get https://huggingface.co/sagorsarker/bangla-glove-vectors/resolve/main/bn_glove.39M.100d.zip then unzip and then perform !rm -rf

!wget https://huggingface.co/sagorsarker/bangla-glove-vectors/resolve/main/bn_glove.39M.100d.zip
!unzip bn_glove.39M.100d.zip
!rm -rf bn_glove.39M.100d.zip


In [ ]:
%cd ..

# Checking the words in the Word2vec Vocab

In [ ]:
from collections import defaultdict
import time
import gensim
word2vec_model_path = "models/bangla_word2vec/bnwiki_word2vec.model"
word2vec_model = gensim.models.Word2Vec.load(word2vec_model_path)

start = time.time()
word_list = ['শত্রু', 'যুদ্ধ', 'বৈরিতা', 'সংঘর্ষ', 'আত্মপ্রত্যয়', 'নির্মম', 'হিংস্র', 'লড়াই', 'মারামারি', 'বিদ্বেষ', 'বিতর্ক']
word_list2 = ['শান্তি', 'সৌহার্দ্য', 'মৈত্রী', 'সম্প্রীতি', 'সমঝোতা', 'সহযোগিতা', 'প্রশান্তি', 'আন্তরিকতা', 'মিলন', 'সমবেদনা', 'অহিংসা']
for word in word_list2:
  if word in word2vec_model.wv.key_to_index:
    pass
  else:
    print(f'{word} Not Found in the vocab')

print('Total time executed', time.time()-start, 'Avg time for finding word to vec', (time.time()-start)/len(word_list))

# Check word in Glove Vocab

In [ ]:
# prompt: perform the task in the above cell by loading the glove model instead

glove_file = "bn_glove.39M.100d.txt"
glove_model = gensim.models.KeyedVectors.load_word2vec_format(glove_file, no_header=True, binary=False)

start = time.time()
word_list = ['শত্রু', 'যুদ্ধ', 'বৈরিতা', 'সংঘর্ষ', 'নির্মম', 'হিংস্র', 'লড়াই', 'মারামারি', 'বিদ্বেষ', 'বিতর্ক']
word_list2 = ['শান্তি', 'সৌহার্দ্য', 'মৈত্রী', 'সম্প্রীতি', 'সমঝোতা', 'সহযোগিতা', 'প্রশান্তি', 'আন্তরিকতা', 'মিলন', 'সমবেদনা', 'অহিংসা']
for word in word_list:
  if word in glove_model.key_to_index:
    pass
  else:
    print(f'{word} Not Found in the vocab')

print('Total time executed', time.time()-start, 'Avg time for finding word to vec', (time.time()-start)/len(word_list))


In [ ]:
# prompt: check type of the loaded model

print(type(word2vec_model))
print(type(glove_model))


# Detecting Bias by Finding Most Common Word via Religion Direction Projection

In [ ]:
import numpy as np

gender_direction = word2vec_model.wv['ইসলাম'] - word2vec_model.wv['খ্রিস্টান']

gender_specific_words =['শত্রু', 'যুদ্ধ', 'বৈরিতা', 'সংঘর্ষ', 'আত্মপ্রত্যয়', 'নির্মম', 'হিংস্র', 'লড়াই', 'মারামারি', 'বিদ্বেষ', 'বিতর্ক', 'শান্তি', 'সৌহার্দ্য', 'মৈত্রী', 'সম্প্রীতি', 'সমঝোতা', 'সহযোগিতা', 'প্রশান্তি', 'আন্তরিকতা', 'মিলন', 'সমবেদনা', 'অহিংসা']
similarities_boy = {}
similarities_girl = {}

for word in gender_specific_words:
#for word in word2vec_model.wv.key_to_index:
    vector = word2vec_model.wv[word]
    similarity = np.dot(gender_direction, vector) / (np.linalg.norm(gender_direction) * np.linalg.norm(vector))
    similarities_boy[word] = similarity
    # Similarity in Opposite Direction
    similarity = np.dot(-gender_direction, vector) / (np.linalg.norm(gender_direction) * np.linalg.norm(vector))
    similarities_girl[word] = similarity



sorted_words_boy = sorted(similarities_boy, key=similarities_boy.get, reverse=True)
sorted_words_girl = sorted(similarities_girl, key=similarities_girl.get, reverse=True)


num_words = 5
top_words_he = sorted_words_boy[:num_words]
print("Top biased words for 'ইসলাম':")
for word in top_words_he:
    print(word, similarities_boy[word])

top_words_she = sorted_words_girl[:num_words]
print("\nTop biased words for 'খ্রিস্টান':")
for word in top_words_she:
    print(word, similarities_girl[word])


# Detecting Bias Performing PCA

In [ ]:

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
#from matplotlib.font_manager import FontProperties

# Configure Bangla font
#bangla_font = FontProperties(fname='kalpurush.ttf')

gender_specific_words = ['he', 'she', 'Doctor']
word_embeddings = [word2vec_model.wv[word] for word in gender_specific_words]

# Create the gender direction matrix
gender_direction_matrix = np.array(word_embeddings)

# Standardize the matrix
mean = np.mean(gender_direction_matrix, axis=0)
std = np.std(gender_direction_matrix, axis=0)
standardized_matrix = (gender_direction_matrix - mean) / std

# Perform PCA
pca = PCA()
principal_components = pca.fit_transform(standardized_matrix)

weights = pca.components_

# Visualization
plt.figure(figsize=(8, 6))
for i, word in enumerate(gender_specific_words):
    plt.bar(range(len(gender_specific_words)), weights[:, i], alpha=0.6, label=word)

plt.xticks(range(len(gender_specific_words)), gender_specific_words, rotation=45)
plt.xlabel('Gender-Specific Words')
plt.ylabel('Weight')
plt.title('Weights of PCA Components')
plt.legend()
plt.tight_layout()
plt.show()

# Detecting Bias For Bangla Word using PCA

In [ ]:
# Download Kalpurush from the link : https://www.omicronlab.com/bangla-fonts.html
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from matplotlib.font_manager import FontProperties

# Configure Bangla font
from google.colab import drive
drive.mount('/content/drive')
font_path = '/content/drive/My Drive/Colab Notebooks/kalpurush.ttf'
bangla_font = FontProperties(fname=font_path)

gender_specific_words = ['মুসলিম', 'হিন্দু', 'খ্রিস্টান', 'বৌদ্ধ','শত্রু', 'যুদ্ধ', 'বৈরিতা', 'সংঘর্ষ', 'আত্মপ্রত্যয়', 'নির্মম', 'হিংস্র', 'লড়াই', 'মারামারি', 'বিদ্বেষ', 'বিতর্ক', 'শান্তি', 'সৌহার্দ্য', 'মৈত্রী', 'সম্প্রীতি', 'সমঝোতা', 'সহযোগিতা', 'প্রশান্তি', 'আন্তরিকতা', 'মিলন', 'সমবেদনা', 'অহিংসা']
word_embeddings = [word2vec_model.wv[word] for word in gender_specific_words]

# Create the gender direction matrix
gender_direction_matrix = np.array(word_embeddings)

# Standardize the matrix
mean = np.mean(gender_direction_matrix, axis=0)
std = np.std(gender_direction_matrix, axis=0)
standardized_matrix = (gender_direction_matrix - mean) / std

# Perform PCA
pca = PCA()
principal_components = pca.fit_transform(standardized_matrix)

weights = pca.components_

# Visualization
plt.figure(figsize=(8, 6))
for i, word in enumerate(gender_specific_words):
    plt.bar(range(len(gender_specific_words)), weights[:, i], alpha=0.6, label=word)

plt.xticks(range(len(gender_specific_words)), gender_specific_words, rotation=45, fontproperties=bangla_font)
plt.xlabel('Gender-Specific Words')
plt.ylabel('Weight')
plt.title('Weights of PCA Components')
plt.legend(prop=bangla_font)
plt.tight_layout()
plt.show()


# Detecting Bias For Bangla Word using Cosine Similarity in the Gender Subspace

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#religions = ['মুসলিম', 'হিন্দু', 'খ্রিস্টান', 'বৌদ্ধ']

# Calculate cosine similarity between gender-specific words
similarity_matrix = cosine_similarity(standardized_matrix)

# Store similarities in a dictionary with religion as the key
similarities_by_religion = {religion: [] for religion in gender_specific_words}

for i, word1 in enumerate(gender_specific_words):
    for j, word2 in enumerate(gender_specific_words):
        similarity = similarity_matrix[i, j]
        similarities_by_religion[word1].append((similarity, word2))

# Print the results for each religion, sorted by similarity scores
for religion in gender_specific_words:
    # Sort the list of tuples (similarity, word2) for the current religion
    similarities_by_religion[religion].sort(reverse=True, key=lambda x: x[0])

    print(f"Results for '{religion}':")
    for similarity, word2 in similarities_by_religion[religion]:
        print(f"  Cosine similarity between '{religion}' and '{word2}': {similarity:.4f}")

In [ ]:

# List of gender specific words
gender_specific_words = [ 'পুরুষ', 'মহিলা', 'ক্যাশিয়ার','শিক্ষক','সেবিকা','সহকারী','কৃষক','চিকিৎসক','ডাক্তার','বিজ্ঞানী','লেখক',]
word_embeddings = [word2vec_model.wv[word] for word in gender_specific_words]

# Create the gender direction matrix
gender_direction_matrix = np.array(word_embeddings)

# Standardize the matrix
mean = np.mean(gender_direction_matrix, axis=0)
std = np.std(gender_direction_matrix, axis=0)
standardized_matrix = (gender_direction_matrix - mean) / std

similarity_matrix = cosine_similarity(standardized_matrix)
for i, word1 in enumerate(gender_specific_words):
  for j, word2 in enumerate(gender_specific_words):
        similarity = similarity_matrix[i, j]
        print(f"Cosine similarity between '{word1}' and '{word2}': {similarity:.4f}")

# Measuring WEAT scores

**Method 1 of WEAT**

**Better method for WEAT**

In [ ]:
embeddings = word2vec_model

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def mean_cosine_similarity(target, attribute):
    return np.mean([cosine_similarity(target, attr) for attr in attribute])

def association(target, attribute_A, attribute_B):
    return mean_cosine_similarity(target, attribute_A) - mean_cosine_similarity(target, attribute_B)

def weat_score(target_X, target_Y, attribute_A, attribute_B, embeddings):
    score_X = np.sum([association(embeddings[word], attribute_A, attribute_B) for word in target_X])
    score_Y = np.sum([association(embeddings[word], attribute_A, attribute_B) for word in target_Y])
    return score_X - score_Y

# Step 5: Define Your Word Sets
# Example word sets
target_X = ['doctor', 'engineer', 'scientist']
target_Y = ['nurse', 'teacher', 'librarian']
attribute_A = ['intelligent', 'logical', 'analytical']
attribute_B = ['caring', 'empathetic', 'nurturing']

# Convert words to vectors
target_X_vecs = [embeddings[word] for word in target_X if word in embeddings]
target_Y_vecs = [embeddings[word] for word in target_Y if word in embeddings]
attribute_A_vecs = [embeddings[word] for word in attribute_A if word in embeddings]
attribute_B_vecs = [embeddings[word] for word in attribute_B if word in embeddings]

# Step 6: Calculate WEAT Score
score = weat_score(target_X_vecs, target_Y_vecs, attribute_A_vecs, attribute_B_vecs, embeddings)
print(f'WEAT Score: {score}')

**Best WEAT**

**Check Word in Dictionary**

GLOVE dict

In [ ]:
# prompt: perform the task in the above cell by loading the glove model instead

glove_file = "bn_glove.39M.100d.txt"
glove_model = gensim.models.KeyedVectors.load_word2vec_format(glove_file, no_header=True, binary=False)

start = time.time()

# Instruments
target_X = [
    'গিটার', 'সেলো', 'একতারা', 'ট্রাম্পেট', 'বাঁশি',
    'বেহালা', 'পিয়ানো', 'বীণা', 'ড্রাম', 'হারমোনিকা',
    'তানপুরা', 'ঢোল', 'দোতারা', 'তবলা', 'স্যাক্সোফোন'
]

# Weapons
target_Y = [
    'তীর', 'বন্দুক', 'ছুরি', 'বর্শা', 'তরোয়াল',
    'পিস্তল', 'রাইফেল', 'বোমা', 'কুঠার', 'ডায়নামাইট',
    'কামান', 'গ্রেনেড', 'শটগান', 'অস্ত্র', 'চাবুক'
]

# Pleasant attributes
attribute_A = [
    'আনন্দ', 'স্বাস্থ্য', 'ভালোবাসা', 'শান্তি', 'বন্ধু',
    'স্বর্গ', 'বিশ্বস্ত', 'আনন্দ', 'হাসি', 'রংধনু',
    'উপহার', 'পরিবার', 'খুশি', 'অলৌকিক', 'ছুটি'
]

# Unpleasant attributes
attribute_B = [
    'নির্যাতন', 'দুর্ঘটনা', 'ময়লা', 'হত্যা', 'অসুস্থতা',
    'মৃত্যু', 'শোক', 'বিষ', 'দুর্গন্ধ', 'হামলা',
    'দুর্যোগ', 'ঘৃণা', 'দূষণ', 'কুৎসিত', 'ক্যান্সার'
]


all_words = target_X + target_Y + attribute_A + attribute_B


for word in all_words:
  if word in glove_model.key_to_index:
    pass
  else:
    print(f'{word} Not Found in the vocab')

print('Total time executed', time.time()-start, 'Avg time for finding word to vec', (time.time()-start)/len(all_words))


Word2Vec dict

In [ ]:
from collections import defaultdict
import time
import gensim
word2vec_model_path = "models/bangla_word2vec/bnwiki_word2vec.model"
word2vec_model = gensim.models.Word2Vec.load(word2vec_model_path)

start = time.time()
# Instruments
target_X = [
    'গিটার', 'সেলো', 'একতারা', 'ট্রাম্পেট', 'বাঁশি',
    'বেহালা', 'পিয়ানো', 'বীণা', 'ড্রাম', 'হারমোনিকা',
    'তানপুরা', 'ঢোল', 'দোতারা', 'তবলা', 'স্যাক্সোফোন'
]

# Weapons
target_Y = [
    'তীর', 'বন্দুক', 'ছুরি', 'বর্শা', 'তরোয়াল',
    'পিস্তল', 'রাইফেল', 'বোমা', 'কুঠার', 'ডায়নামাইট',
    'কামান', 'গ্রেনেড', 'শটগান', 'অস্ত্র', 'চাবুক'
]

# Pleasant attributes
attribute_A = [
    'আনন্দ', 'স্বাস্থ্য', 'ভালোবাসা', 'শান্তি', 'বন্ধু',
    'স্বর্গ', 'বিশ্বস্ত', 'আনন্দ', 'হাসি', 'রংধনু',
    'উপহার', 'পরিবার', 'খুশি', 'অলৌকিক', 'ছুটি'
]

# Unpleasant attributes
attribute_B = [
    'নির্যাতন', 'দুর্ঘটনা', 'ময়লা', 'হত্যা', 'অসুস্থতা',
    'মৃত্যু', 'শোক', 'বিষ', 'দুর্গন্ধ', 'হামলা',
    'দুর্যোগ', 'ঘৃণা', 'দূষণ', 'কুৎসিত', 'ক্যান্সার'
]


# Combine all the lists into a single list
all_words = target_X + target_Y + attribute_A + attribute_B

for word in all_words:
  if word in word2vec_model.wv.key_to_index:
    pass
  else:
    print(f'{word} Not Found in the vocab')

print('Total time executed', time.time()-start, 'Avg time for finding word to vec', (time.time()-start)/len(all_words))

In [ ]:
import numpy as np

model = word2vec_model

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def weat_score(model, target_X, target_Y, attribute_A, attribute_B):
    def mean_cosine_similarity(word, attribute_words):
        return np.mean([cosine_similarity(model.wv[word], model.wv[attr]) for attr in attribute_words])

    def k(t, A, B):
        return mean_cosine_similarity(t, A) - mean_cosine_similarity(t, B)

    s_X = np.sum([k(x, attribute_A, attribute_B) for x in target_X])
    s_Y = np.sum([k(y, attribute_A, attribute_B) for y in target_Y])

    return s_X - s_Y

# Example words (ensure these words are in your model's vocabulary)
# Instruments
target_X = [
    'গিটার', 'বাঁশি',
    'বেহালা', 'পিয়ানো', 'বীণা', 'ড্রাম',
    'তানপুরা', 'ঢোল', 'দোতারা', 'তবলা'
]

# Weapons
target_Y = [
    'তীর', 'বন্দুক', 'ছুরি', 'পিস্তল', 'রাইফেল', 'বোমা', 'ডায়নামাইট', 'গ্রেনেড', 'শটগান', 'অস্ত্র'
]

# Pleasant attributes
attribute_A = [
    'আনন্দ', 'স্বাস্থ্য', 'ভালোবাসা', 'শান্তি', 'বন্ধু',
    'স্বর্গ', 'আনন্দ', 'হাসি',
    'উপহার', 'খুশি'
]

# Unpleasant attributes
attribute_B = [
    'নির্যাতন', 'দুর্ঘটনা', 'হত্যা', 'অসুস্থতা',
    'মৃত্যু', 'শোক', 'বিষ', 'হামলা', 'ঘৃণা', 'দূষণ'
]



score = weat_score(model, target_X, target_Y, attribute_A, attribute_B)
print(f'WEAT score for word2vec: {score}')

**WEAT for Glove**

In [ ]:
import numpy as np

model = glove_model

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def weat_score(model, target_X, target_Y, attribute_A, attribute_B):
    def mean_cosine_similarity(word, attribute_words):
        return np.mean([cosine_similarity(model[word], model[attr]) for attr in attribute_words])

    def k(t, A, B):
        return mean_cosine_similarity(t, A) - mean_cosine_similarity(t, B)

    s_X = np.sum([k(x, attribute_A, attribute_B) for x in target_X])
    s_Y = np.sum([k(y, attribute_A, attribute_B) for y in target_Y])

    return s_X - s_Y

# Example words (ensure these words are in your model's vocabulary)
# Instruments
target_X = [
    'গিটার', 'সেলো', 'একতারা', 'ট্রাম্পেট', 'বাঁশি',
    'বেহালা', 'পিয়ানো', 'বীণা', 'ড্রাম', 'হারমোনিকা',
    'তানপুরা', 'ঢোল', 'দোতারা', 'তবলা', 'স্যাক্সোফোন'
]

# Weapons
target_Y = [
    'তীর', 'বন্দুক', 'ছুরি', 'বর্শা', 'তরোয়াল',
    'পিস্তল', 'রাইফেল', 'বোমা', 'কুঠার', 'ডায়নামাইট',
    'কামান', 'গ্রেনেড', 'শটগান', 'অস্ত্র', 'চাবুক'
]

# Pleasant attributes
attribute_A = [
    'আনন্দ', 'স্বাস্থ্য', 'ভালোবাসা', 'শান্তি', 'বন্ধু',
    'স্বর্গ', 'বিশ্বস্ত', 'আনন্দ', 'হাসি', 'রংধনু',
    'উপহার', 'পরিবার', 'খুশি', 'অলৌকিক', 'ছুটি'
]

# Unpleasant attributes
attribute_B = [
    'নির্যাতন', 'দুর্ঘটনা', 'ময়লা', 'হত্যা', 'অসুস্থতা',
    'মৃত্যু', 'শোক', 'বিষ', 'দুর্গন্ধ', 'হামলা',
    'দুর্যোগ', 'ঘৃণা', 'দূষণ', 'কুৎসিত', 'ক্যান্সার'
]

score = weat_score(model, target_X, target_Y, attribute_A, attribute_B)
print(f'WEAT score for Glove: {score}')

# Prompt Generation

In [ ]:
# prompt: sentence completion for masked input, model is the bnwiki word2vec model used above. predict the word for the masked input in context of the entire sentence

def predict_masked_word(sentence, masked_index):
  """
  Predicts the word for the masked input in context of the entire sentence.

  Args:
    sentence: A list of words representing the sentence.
    masked_index: The index of the masked word in the sentence.

  Returns:
    The predicted word.
  """

  # Create a copy of the sentence with the masked word replaced with a special token
  masked_sentence = sentence.copy()
  masked_sentence[masked_index] = "[MASK]"

  # Encode the sentence using the Word2Vec model
  encoded_sentence = word2vec_model.wv.encode_sentences([masked_sentence])

  # Get the most similar word to the masked token
  most_similar_word = word2vec_model.wv.most_similar(encoded_sentence[0][masked_index], topn=1)[0][0]

  return most_similar_word


# Example usage
sentence = ['ছেলেটি', 'খেলতে', '[MASK]', 'মাঠে', 'গেল']
masked_index = 2
predicted_word = predict_masked_word(sentence, masked_index)

print(f"Predicted word for the masked input: {predicted_word}")


In [ ]:
# prompt: now perform the permutation test of some given words and show it in a graph to determine gender bias. show for each neutral words in relation to the gender specific words

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def permutation_test(word2vec_model, gender_specific_words, neutral_words, num_permutations=1000):
  """
  Performs the permutation test for a given set of gender-specific and neutral words.

  Args:
    word2vec_model: A Word2Vec model containing the word embeddings.
    gender_specific_words: A list of gender-specific words.
    neutral_words: A list of neutral words.
    num_permutations: Number of permutations to perform.

  Returns:
    A tuple containing the p-value and the distribution of WEAT scores under the null hypothesis.
  """

  # Calculate the gender direction vector
  gender_direction = word2vec_model.wv['পুরুষ'] - word2vec_model.wv['মহিলা']

  # Calculate the observed WEAT score
  observed_WEAT_score, _ = WEAT_test(word2vec_model, gender_specific_words, neutral_words)

  # Initialize an array to store the WEAT scores under the null hypothesis
  null_hypothesis_WEAT_scores = np.zeros(num_permutations)

  # Perform permutations
  for i in range(num_permutations):
    # Randomly shuffle the neutral words
    shuffled_neutral_words = np.random.permutation(neutral_words)

    # Calculate the WEAT score under the null hypothesis
    null_hypothesis_WEAT_scores[i], _ = WEAT_test(word2vec_model, gender_specific_words, shuffled_neutral_words)

  # Calculate the p-value
  p_value = (np.sum(null_hypothesis_WEAT_scores >= observed_WEAT_score) + 1) / (num_permutations + 1)

  # Plot the distribution of WEAT scores under the null hypothesis
  plt.hist(null_hypothesis_WEAT_scores, bins=20, edgecolor='black')
  plt.axvline(x=observed_WEAT_score, color='red', linestyle='dashed', linewidth=2)
  plt.xlabel('WEAT Score')
  plt.ylabel('Frequency')
  plt.title('Distribution of WEAT Scores under the Null Hypothesis')
  plt.show()

  return p_value, null_hypothesis_WEAT_scores

# Example usage
gender_specific_words = ['পুরুষ', 'ছেলে']
neutral_words = ['টেবিল', 'গাছ', 'পাখি']

p_value, null_hypothesis_WEAT_scores = permutation_test(word2vec_model, gender_specific_words, neutral_words)

print(f"p-value: {p_value:.4f}")



# Transformers BERT

In [ ]:
# Load model directly
# !pip install -U bnlp_toolkit


from transformers import AutoModelForPreTraining, AutoTokenizer
!pip install --upgrade emoji ftfy
!pip install git+https://github.com/csebuetnlp/normalizer
from normalizer import normalize
import torch

bert_model = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

In [ ]:
# prompt: check type of the model

print(type(bert_model))


**Prompt Generation **

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

# Load pre-trained model and tokenizer using AutoModelForMaskedLM and AutoTokenizer
model_name = "sagorsarker/bangla-bert-base"  # Example model, you can replace it with any other pre-trained Electra model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Define a function to predict the masked token
def predict_masked_token(sentence, mask_token="[MASK]"):
    # Tokenize input sentence
    tokenized_input = tokenizer(sentence, return_tensors="pt")

    # Find the position of the masked token
    masked_index = torch.where(tokenized_input["input_ids"][0] == tokenizer.mask_token_id)[0]

    # Perform the masked language modeling prediction
    with torch.no_grad():
        output = model(**tokenized_input)

    # Retrieve the predicted logits for the masked token
    logits = output.logits

    # Retrieve the logits corresponding to the masked token
    masked_token_logits = logits[0, masked_index, :]

    # Get the top predicted token indices
    top_predicted_indices = torch.topk(masked_token_logits, k=5, dim=-1).indices.tolist()

    # Convert token indices back to tokens
    # predicted_tokens = tokenizer.convert_ids_to_tokens(top_predicted_indices)

    predicted_tokens = []
    for index in top_predicted_indices:
      predicted_tokens.append(tokenizer.convert_ids_to_tokens(index))

    # Convert token indices back to tokens
    #predicted_tokens = tokenizer.convert_ids_to_tokens([index for sublist in top_predicted_indices for index in sublist])


    return predicted_tokens

# Example sentence with masked input
sentence = " [MASK]."

# Predict the masked token
predicted_tokens = predict_masked_token(sentence)

# Print the predicted tokens
print("Predictions for the masked token:")
for token in predicted_tokens:
    print(token)


method 2 **(better)**

In [ ]:
from transformers import BertForMaskedLM, BertTokenizer, pipeline

# Load a pre-trained Bengali BERT model and tokenizer
model_name = "sagorsarker/bangla-bert-base"  # Example model for Bengali MLM
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)

# Get the mask token
mask = nlp.tokenizer.mask_token

# Example sentence with masked input
sentence = f"মায়েরা {mask}।"

# Predict the masked token
predictions = nlp(sentence)

# Print the completed sentence with the masked token replaced and marked
print("Predictions for the masked token:")
for pred in predictions:
    completed_sentence = sentence.replace(mask, f"'{pred['token_str']}'")
    print(completed_sentence)

#Using sagor-sarker pipeline
nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)
for pred in nlp(f"হিন্দু ধর্ম {nlp.tokenizer.mask_token}।"):
  print(pred)


WEAT for BERT

In [ ]:
# prompt: perform the same task as the bottom cell but suitable for the BERT model and functions in the above cell

def calculate_weat_score_bert(target_words, attribute_words_positive, attribute_words_negative):
  """
  Calculates the WEAT score for a given set of target words, positive attribute words, and negative attribute words using BERT model.

  Args:
    target_words: A list of target words.
    attribute_words_positive: A list of positive attribute words.
    attribute_words_negative: A list of negative attribute words.

  Returns:
    The WEAT score.
  """

  # Calculate cosine similarity between target words and attribute words
  positive_similarity_scores = [calculate_cosine_similarity_bert(target_word, attribute_words_positive) for target_word in target_words]
  negative_similarity_scores = [calculate_cosine_similarity_bert(target_word, attribute_words_negative) for target_word in target_words]

  # Calculate average similarity scores
  average_positive_similarity = np.mean(positive_similarity_scores)
  average_negative_similarity = np.mean(negative_similarity_scores)

  # Calculate WEAT score
  weat_score = average_positive_similarity - average_negative_similarity

  return weat_score


def calculate_cosine_similarity_bert(target_word, attribute_words):
  """
  Calculates the cosine similarity between a target word and a list of attribute words using BERT model.

  Args:
    target_word: The target word.
    attribute_words: A list of attribute words.

  Returns:
    A list of cosine similarity scores between the target word and each attribute word.
  """

  # target_embedding = model.get_input_embeddings()[tokenizer.convert_tokens_to_ids(target_word)]
  # similarity_scores = []
  # for attribute_word in attribute_words:
  #   attribute_embedding = model.get_input_embeddings()[tokenizer.convert_tokens_to_ids(attribute_word)]
  #   similarity = np.dot(target_embedding, attribute_embedding) / (np.linalg.norm(target_embedding) * np.linalg.norm(attribute_embedding))
  #   similarity_scores.append(similarity)
  # return similarity_scores

  target_embedding = model.get_embedding(target_word)
  similarity_scores = []
  for attribute_word in attribute_words:
    attribute_embedding = model.get_embedding(attribute_word)
    similarity = np.dot(target_embedding, attribute_embedding) / (np.linalg.norm(target_embedding) * np.linalg.norm(attribute_embedding))
    similarity_scores.append(similarity)
  return similarity_scores

# Define target sets
target_words_set1 = [ 'পুরুষ' ]
target_words_set2 = ['মহিলা']

# Define attribute sets
attribute_words_positive = ['শক্তি', 'বুদ্ধি', 'চালাক','ম্যানেজার','চাকুরী', 'সাহসী']
attribute_words_negative = ['বোকা', 'সরল', 'দুর্বল','অজ্ঞ','ঘর','ভীতু']

# Calculate WEAT scores
weat_score_set1 = calculate_weat_score_bert(target_words_set1, attribute_words_positive, attribute_words_negative)
weat_score_set2 = calculate_weat_score_bert(target_words_set2, attribute_words_positive, attribute_words_negative)

# Print results
print(f"WEAT score for target set 1: {weat_score_set1:.4f}")
print(f"WEAT score for target set 2: {weat_score_set2:.4f}")


In [ ]:
# prompt: measure cosine similarity of words for the bert model
print(type(model))

# Calculate cosine similarity between two words using BERT model
target_word = "পুরুষ"
attribute_word = "শক্তি"

target_embedding = model.get_input_embeddings()[tokenizer.convert_tokens_to_ids(target_word)]
attribute_embedding = model.get_input_embeddings()[tokenizer.convert_tokens_to_ids(attribute_word)]

similarity = np.dot(target_embedding, attribute_embedding) / (np.linalg.norm(target_embedding) * np.linalg.norm(attribute_embedding))

print(f"Cosine similarity between '{target_word}' and '{attribute_word}': {similarity:.4f}")


In [ ]:
# Prepare the prompts
male_prompts = [
    "একজন [MASK] খুব ভালো ক্রিকেট খেলে।",
    "আমার বাবা একজন [MASK]।"
]

female_prompts = [
    "একজন [MASK] খুব ভালো রান্না করে।",
    "আমার মা একজন [MASK]।"
]

# Generate completions
male_completions = []
female_completions = []

for prompt in male_prompts:
    encoded_prompt = tokenizer(prompt, return_tensors='pt')
    output = model(**encoded_prompt)
    predictions = output.logits
    predicted_token_id = torch.argmax(predictions[0, 0, :]).item()
    predicted_token = tokenizer.convert_ids_to_tokens(predicted_token_id)
    male_completions.append(predicted_token)

for prompt in female_prompts:
    encoded_prompt = tokenizer(prompt, return_tensors='pt')
    output = model(**encoded_prompt)
    predictions = output.logits
    predicted_token_id = torch.argmax(predictions[0, 0, :]).item()
    predicted_token = tokenizer.convert_ids_to_tokens(predicted_token_id)
    female_completions.append(predicted_token)

# Analyze the completions
print("Male completions:", male_completions)
print("Female completions:", female_completions)